In [10]:
import pandas as pd
from collections import Counter
import math
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import IncrementalPCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
import matplotlib.pyplot as plt

In [11]:
from tensorflow.keras import layers, models
import tensorflow as tf

In [12]:
data = pd.read_csv('../data/fake reviews dataset.csv')
o_r = data[data['label'] == 'OR']
cg = data[data['label'] == 'CG']

In [13]:
print('Number of original reviews: ', len(o_r))
print('Number of computer generated reviews: ', len(cg))

Number of original reviews:  20216
Number of computer generated reviews:  20216


In [16]:
def calculate_tfidf(reviews):
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(reviews)
    tfidf_scores = dict(zip(vectorizer.get_feature_names_out(), tfidf_matrix.sum(axis=0).A1))
    return tfidf_scores

# Calculate TF-IDF for CG and OR reviews separately
cg_tfidf = calculate_tfidf(cg['text_'])
or_tfidf = calculate_tfidf(o_r['text_'])

In [17]:
cg_df = pd.DataFrame(list(cg_tfidf.items()), columns=['word', 'cg_tfidf']).set_index('word')
or_df = pd.DataFrame(list(or_tfidf.items()), columns=['word', 'or_tfidf']).set_index('word')


In [18]:
combined_df = cg_df.join(or_df, how='outer').fillna(0)

combined_df['cg_or_diff'] = combined_df['cg_tfidf'] - combined_df['or_tfidf']

distinctive_words = combined_df.sort_values('cg_or_diff', ascending=False)

top_cg_words = distinctive_words[distinctive_words['cg_or_diff'] > 0].head(10)
top_or_words = distinctive_words[distinctive_words['cg_or_diff'] < 0].head(10)

print("Top distinctive words in CG reviews:")
print(top_cg_words[['cg_tfidf', 'or_tfidf', 'cg_or_diff']])

print("\nTop distinctive words in OR reviews:")
print(top_or_words[['cg_tfidf', 'or_tfidf', 'cg_or_diff']])

Top distinctive words in CG reviews:
              cg_tfidf    or_tfidf  cg_or_diff
word                                          
book        855.418973  421.821671  433.597302
story       585.070164  262.807302  322.262862
characters  462.646307  141.606533  321.039773
great       819.893869  508.196966  311.696903
good        703.649544  400.381030  303.268514
loves       460.073096  164.873049  295.200048
movie       466.431590  176.407040  290.024549
read        581.216130  295.899410  285.316720
little      476.424847  241.376555  235.048292
bought      432.204544  202.482418  229.722126

Top distinctive words in OR reviews:
              cg_tfidf  or_tfidf  cg_or_diff
word                                        
racer         0.431879  0.432281   -0.000403
stylized      0.179614  0.180701   -0.001086
bookcase      0.196276  0.198093   -0.001817
cookbook      1.381391  1.383275   -0.001885
rex           1.163751  1.165808   -0.002057
fest          0.383117  0.385333   -0.002217
d